In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [18]:
r = requests.get('http://localhost:8000/products?format=json')
shop_df = pd.DataFrame(json.loads(r.content.decode()))
shop_df['name'] = [i.lower() for i in shop_df['name']]
shop_df.head()

In [66]:
filepath = 'shopping_lists/shoppingList13.json'
with open(filepath) as f:
    shopping_list = json.load(f)

In [62]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    return items_df

In [63]:
items_df = get_items(shopping_list)

In [64]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,600537,"[artichoke, hearts]",Canned and Jarred,93828,79.4,g
1,600547,"[green, bell, pepper]",Produce,11333,1.0,
2,600573,"[red, pepper, flakes]",Spices and Seasonings,1032009,0.1,g
3,600587,"[chicken, breasts]",Meat,5062,1.0,
4,600591,"[maple, syrup]",Cereal,19911,16.7,ml
5,600593,"[frozen, spinach]",Frozen,11463,45.4,g
6,600597,"[parmesan, cheese]",Cheese,1033,1.0,
7,600601,"[salt, and, pepper]",Pantry Items,1102047,4.0,
8,600613,[naan],Bakery/Bread,93723,3.0,pieces
9,600617,[rice],Pasta and Rice,20444,138.8,g


In [65]:
matches_df = pd.DataFrame()
for product in items_df.iloc:
    regex = ''.join([f'(?=.*{i})' for i in product['name']])   
    entries = shop_df['name'].str.contains(regex)    
    if entries.any():
        matches = shop_df[entries]
        print(f"appending {' '.join(product['name'])}")
        matches_df = matches_df.append(matches)
    else:
        print(f"no such product: {' '.join(product['name'])}")

no such product: artichoke hearts
no such product: green bell pepper
no such product: red pepper flakes
appending chicken breasts
no such product: maple syrup
no such product: frozen spinach
appending parmesan cheese
no such product: salt and pepper
appending naan
appending rice
no such product: pecan
appending milk
no such product: dried cranberries
no such product: evaporated milk


In [59]:
shop_df[shop_df['name'].str.contains('(?=.*)(?=.*penne)')]

,store,name,img_url,price,currency,amount,unit
462,sainsburys,sainsbury's fresh egg penne 500g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.75,GBP,500g,unit
724,sainsburys,sainsbury's penne 1kg,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.00,GBP,1kg,unit
821,sainsburys,napolina penne 500g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.30,GBP,500g,unit


### add to the database:
    - ground cumin / cumin
    - hot sauce
    - almonds, agave
    - salsa
    - tahini
    - almond milk
    - worcestershire sauce
    - flax egg
    - maple syrup
    - lemon juice
    - garlic powder
    - gluten-free
    - lime juice
    - pesto
    - mozzarella
    - cornstarch
    - green bell pepper
    - red pepper flakes
    - pumpking seeds
    - dressings (caeasar etc)
    - cashews
    - dijon mustard
    - chili sauce
    - oats
    - peanut butter
    - tortilas
    - vanilla extract
    - starch
    - vinegar (white, apple cider)

### possibly drop stuff like 
    - quick 
    - canned
    - cooking 
    - fat
    - ground
    - leaves?
    - plain
    - cooked
    - pasta if say 'penne pasta'
    - full-fat
    - if ground and meat then mince
    - pieces
    - scrambled
    - frozen
    - creamy

### change

- convert 4x5 to 20 etc
- convert price to one common unit (pence)

In [287]:
matches_df.sample(frac=0.2)

,store,name,img_url,price,currency,amount,unit
847,sainsburys,kitkat 4 finger milk chocolate bar multipack 4...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.00,GBP,4x41.5g,unit
720,sainsburys,cadbury dairy milk chocolate bar 200g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,200g,unit
723,sainsburys,cadbury dairy milk giant chocolate buttons bag...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,119g,unit
901,sainsburys,cadbury dairy milk chocolate bar 10x29.3g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.00,GBP,10x29.3g,unit
1089,sainsburys,"sainsbury's indulgence ice cream, milk chocola...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.35,GBP,4x110ml,unit
702,sainsburys,mcvitie's milk chocolate digestives biscuits 266g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.60,GBP,266g,unit
632,sainsburys,sainsbury's toastie thick sliced white bread 800g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,55.00,p,800g,unit


In [68]:
def match_product(name, price, quantity, unit):
    # TODO
    ...